读取clinvar数据

In [ ]:
import pandas as pd

file_path = r'D:\data\clinvar\variant_summary.txt\variant_summary.txt'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
pathogenicity_levels = ['Benign', 'Benign/Likely benign', 'Likely benign', 'Likely pathogenic', 'Pathogenic']
variants_by_pathogenicity = []
for patho in pathogenicity_levels:
    variants_by_pathogenicity.append(df['ClinicalSignificance'].value_counts()[patho])
variants_by_pathogenicity

In [ ]:
chrs = [str(i) for i in range(1, 23)]
variants_chr_counts = df['Chromosome'].value_counts()
chrs.extend(['X', 'Y', 'MT'])
variants_by_chr = []
for chr in chrs:
    print(chr)
    variants_by_chr.append(variants_chr_counts[chr])
variants_by_chr

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 3))
plt.subplot(121)
plt.bar(pathogenicity_levels, variants_by_pathogenicity)
plt.subplot(122)
plt.bar(chrs, variants_by_chr)
plt.savefig('../img/data_desc/raw_data_description.png')
plt.show()

In [ ]:
df['Type'].value_counts()

In [ ]:
import pandas as pd

file_path = r'D:\data\clinvar\variant_summary.txt\variant_summary.txt'
df = pd.read_csv(file_path, delimiter='\t')
df = df[df['Type']=='single nucleotide variant']
df = df[(df['ClinicalSignificance'] == 'Pathogenic') | (df['ClinicalSignificance'] == 'Benign')]
df = df[df['Assembly'] == 'GRCh38']
df = df[df['Chromosome']!='MT']
df['label'] = -1
df['label'][df['ClinicalSignificance'] == 'Pathogenic'] = 1
df['label'][df['ClinicalSignificance'] != 'Pathogenic'] = 0
df = df[df['ReferenceAlleleVCF'] != 'na']
df.to_csv('../data/variants.csv', index=False)

获取突变位点周围序列

In [ ]:
import pandas as pd

# 取突变位点周围50bp的内容
surrounding_lens = [10000]
df = pd.read_csv('../data/variants.csv')
for chr, subset in df.groupby('Chromosome'):
    print(chr)
    with open(r'../data/ucsc/hg38/chr{}.txt'.format(chr)) as f:
        seq = f.readline()
        for index, variant in subset.iterrows():
            point_index = int(variant['PositionVCF'])
            # point_index = seq[point_index-1]
            ref = variant['ReferenceAlleleVCF']
            alt = variant['AlternateAlleleVCF']
            for surrounding_len in surrounding_lens:
                left = seq[point_index-1-surrounding_len:point_index-1]
                right = seq[point_index: point_index+surrounding_len]
                # print(point_index, ref, alt, seq[point_index-1], left, right)
                if len(left)<surrounding_len:
                    left = 'N' *(surrounding_len-len(left)) + left
                if len(right)<surrounding_len:
                    right = right + 'N'*(surrounding_len-len(right))
                t1 = ''.join([left, ref, right])
                t2 = ''.join([left, alt, right])
                ref_col = 'seq_{}_ref'.format(surrounding_len*2+1)
                alt_col = 'seq_{}_alt'.format(surrounding_len)
                df.loc[index, ref_col] = t1.upper()
                df.loc[index, alt_col] = t1.upper()

In [1]:
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

chrs = [str(i) for i in range(1, 23)]
chrs.extend(['X', 'Y'])
chr_seqs = {}
for chr in chrs:
    with open(r'../data/ucsc/hg38/chr{}.txt'.format(chr)) as f:
        seq = f.readline()
        chr_seqs[chr] = seq 

print("seq loaded")

def get_seq(row, seq_len):
    chr = row['Chromosome']
    # with open(r'../data/ucsc/hg38/chr{}.txt'.format(chr)) as f:
    #     seq = f.readline()
    seq = chr_seqs[chr]
    point_index = int(row['PositionVCF'])
    ref = row['ReferenceAlleleVCF']
    alt = row['AlternateAlleleVCF']
    left = seq[point_index-1-seq_len:point_index-1]
    right = seq[point_index: point_index+seq_len]
    # print(point_index, ref, alt, seq[point_index-1], left, right)
    if len(left)<seq_len:
        left = 'N' *(seq_len-len(left)) + left
    if len(right)<seq_len:
        right = right + 'N'*(seq_len-len(right))
    t1 = ''.join([left, ref, right])
    t2 = ''.join([left, alt, right])
    return t1.upper(), t2.upper()
    
df = pd.read_csv('../data/variants.csv')
df[['seq_20001_ref', 'seq_20001_alt']] = df.parallel_apply(get_seq, result_type='expand', axis=1, seq_len=10000)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
seq loaded


In [ ]:
df[['label', 'seq_20001_ref', 'seq_20001_alt']].to_csv('variants_20001.csv', index=True)

In [ ]:
len(df.iloc[0,]['seq_20001_ref_encoded'])

In [ ]:
df.head()

In [2]:
df = df.rename(columns={"ReferenceAlleleVCF": "Ref", "AlternateAlleleVCF": "Alt", 'seq_20001_ref': 'seq', 'seq_20001_ref_encoded': 'seq_onehot'})

In [5]:
df_selected = df.loc[:,['label', 'Ref', 'Alt', 'seq']]
# df_selected = df[['label', 'Ref']]

In [7]:
df_selected.head()

,label,Ref,Alt,seq
0,1,C,T,AGAATCTGCACCAGGGACTCCTGTAGGGCTTGCTTTACTGTGGAAG...
1,1,A,G,TAAATTGATTTTAGAGAAGGCAGGTCTCTCACCGAAGGAGCCGCAG...
2,1,G,C,CCATGTTTGAAAAATGGTTCCGAGTGCTAAGAAGATTTAAAACTCT...
3,0,G,A,TTGCATCTACGTAGCTCTCACCCCATTTCTTTCCACAACACACATT...
4,1,A,C,CTGCCTCCAAAGAAAGAAAAAGTAAAAGCTAAAAGGCAGAAATGAA...


In [8]:
df_selected.to_csv('variants_onehot.csv', index=False)

In [4]:
def embedding(seq):
        dict_ = {
            'A': [1, 0, 0, 0],
            'T': [0, 1, 0, 0],
            'C': [0, 0, 1, 0],
            'G': [0, 0, 0, 1],
            'N': [0, 0, 0, 0]
        }
        def encode(n):
            if n in dict_.keys():
                return dict_[n]
            else:
                return dict_['N']
        return list(map(encode, list(seq)))

df['seq_onehot'] = df['seq'].parallel_apply(embedding)
# df['seq_20001_alt_encoded'] = df['seq_20001_alt'].parallel_apply(embedding)

KeyboardInterrupt: 

In [ ]:
df['seq_20001_alt_encoded'] = df['seq_20001_alt'].parallel_apply(embedding)

In [ ]:
df['seq_20001_ref_encoded'].to_csv('test.csv', index=False)

In [ ]:
df.to_csv('../data/variants_with_seq.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("../data/variants_with_seq.csv")
df.columns

In [ ]:
df_model = df[['Chromosome', 'ReferenceAlleleVCF', 'AlternateAlleleVCF', 'Start', 'Stop','label',
       'seq_50_ref', 'seq_50_alt', 'seq_100_ref', 'seq_100_alt', 'seq_300_ref',
       'seq_300_alt', 'seq_500_ref', 'seq_500_alt', 'seq_1000_ref',
       'seq_1000_alt']]

In [ ]:
df_model.to_csv('../data/variants_model.csv', index=False)

In [ ]:
df = df[df['ReferenceAlleleVCF'] != 'na']

In [ ]:
df.to_csv('../data/variants_with_seq.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('../data/variants_with_seq.csv')

In [ ]:
pathogenicity_levels = ['Benign', 'Pathogenic']
variants_by_pathogenicity = []
for patho in pathogenicity_levels:
    print(patho)
    variants_by_pathogenicity.append(df['ClinicalSignificance'].value_counts()[patho])
variants_by_pathogenicity

In [ ]:
chrs = [str(i) for i in range(1, 23)]
variants_chr_counts = df['Chromosome'].value_counts()
chrs.extend(['X', 'Y'])
variants_by_chr = []
for chr in chrs:
    variants_by_chr.append(variants_chr_counts[chr])
variants_by_chr

In [ ]:
df.head(1)

In [9]:
df = df_selected

In [10]:
df_positive = df[df['label'] == 1]
df_negative = df[df['label'] == 0]

In [11]:
print(df_positive.shape, df_negative.shape)

(51331, 4) (155863, 4)


In [12]:
num_p = df_positive.shape[0]

In [13]:
num_n = df_negative.shape[0]

In [14]:
train_ratio = 0.7
valid_ration = 0.2
test_ration = 0.1

In [15]:
df_positive_train = df_positive.iloc[:int(num_p*train_ratio)]
df_positive_valid = df_positive.iloc[int(num_p*train_ratio):int(num_p*0.9)]
df_positive_test = df_positive.iloc[int(num_p*0.9):]

In [16]:
print(df_positive_train.shape, df_positive_valid.shape, df_positive_test.shape)

(35931, 4) (10266, 4) (5134, 4)


In [17]:
df_negative_train = df_negative.iloc[:int(num_n*train_ratio)]
df_negative_valid = df_negative.iloc[int(num_n*train_ratio):int(num_n*0.9)]
df_negative_test = df_negative.iloc[int(num_n*0.9):]

In [18]:
print(df_negative_train.shape, df_negative_valid.shape, df_negative_test.shape)

(109104, 4) (31172, 4) (15587, 4)


In [19]:
df_train = pd.concat([df_positive_train, df_negative_train])
df_valid = pd.concat([df_positive_valid, df_negative_valid])
df_test = pd.concat([df_positive_test, df_negative_test])

In [20]:
print(df_train.shape, df_valid.shape, df_test.shape)

(145035, 4) (41438, 4) (20721, 4)


In [ ]:
df_train.to_csv('../data/train_20001_onehot.csv', index=False)
df_valid.to_csv('../data/valid_20001_onehot.csv', index=False)
df_test.to_csv('../data/test_20001_onehot.csv', index=False)